In [19]:
import sys
sys.path.insert(0, '..')
print(sys.path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
print(sys.path)
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


['..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/tmp/tmp9u_12xil', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model']
['..', '..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubu

In [20]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_map.csv")


In [21]:
clean_df.head()

,Unnamed: 0,caption,product
0,0,Tanabata Valentine's Day Shenzhen Bao'an M Nan...,O1CN01cSoTwD1spJos7ZSF6_!!0-item_pic.jpg
1,1,Children's Toys Little Girl over 6 Years Old G...,O1CN01iI5sGv1vIkV5dfICu_!!0-item_pic.jpg
2,2,sm qing qu Alternative Sex between Men and Wom...,O1CN01xnnyaz248W0n5le2q_!!131027346.jpg
3,3,Traditional Chinese Painting Burnin' Up Yingke...,O1CN01lfHuuA1D3K2MEM63p_!!160-0-lubanu.jpg
4,4,lgnace Lee Men's Jeans Thick Section Distresse...,TB23FV3afBNTKJjy0FdXXcPpVXa_!!1944606990.jpg


In [22]:
clean_df.tail()

,Unnamed: 0,caption,product
49995,49995,"The Office for the Narrow Bed off K Frame If, ...",TB1hHR5JW61gK0jSZFlq6xDKFXa
49996,49996,Thick Aluminum xue ping guo F Milk Pot Wooden ...,TB1cVeRzkL0gK0jSZFtq6xQCXXa
49997,49997,Thick Warm Crystal Velvet Quilted Bed Skirt Pi...,O1CN01r4aAWL1Cluyu7h3kf_!!122-0-lubanu.jpg
49998,49998,Plus Velvet Thickening Universal Elastic Wood ...,TB13sAjLAL0gK0jSZFAq6AA9pXa
49999,49999,M2 SKH51 M35 SKH55 M42 High-Speed Steel Bar fd...,O1CN01Q4ztF71RM0mnnKpXE_!!2200742622096.jpg


In [23]:
# col_to_test =  [c for c in clean_df.columns.values if "caption" in c] + [c for c in clean_df.columns.values if "step" in c]
col_to_test =  "caption"

In [24]:
image_size = 384
preprocess = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 


In [25]:
def gen_embed(caption_col,clip_model):
    test_loader,df = prepare_data("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_map.csv",
    CONFIG.test_image_data_dir,"val_imgs"
    ,"product",caption_col,160,preprocess,CONFIG.global_random_state,test=True,tokenize=False)
    trainer = Trainer(accelerator="gpu", devices=[2])
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred

    return image_embed,text_embed

In [26]:
def save_embed(image_embed,text_embed,model_size:str,prefix:str):
    img_embed_path = "./image_embeds/val_set_embed"
    txt_embed_path = "./text_embeds/val_set_embed"
    img_embed_filename = f"img_emded-{model_size}-{prefix}.pkl"
    txt_embed_filename = f"txt_emded-{model_size}-{prefix}.pkl"
    img_embed_full_path = f"{img_embed_path}/{img_embed_filename}"
    txt_embed_full_path = f"{txt_embed_path}/{txt_embed_filename}"

    image_embed_info_dict = {"model_name":prefix,"model_size":model_size,"pair_name":txt_embed_filename,"embed":image_embed}
    text_embed_info_dict = {"model_name":prefix,"model_size":model_size,"pair_name":img_embed_filename,"embed":text_embed}

    with open(img_embed_full_path,"wb") as file:
        print(f"saving img embed from {prefix} to {img_embed_full_path}")
        pickle.dump(image_embed_info_dict,file)
    with open(txt_embed_full_path,"wb") as file:
        print(f"saving txt embed from {prefix} to {txt_embed_full_path}")
        pickle.dump(text_embed_info_dict,file)
    

In [27]:
# models = [
#     {"path":"/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/save_checkpoint_7_thu.pth",
#     "model_size":"base","model_name":"base_all_e_7"},
#     {"path":"/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model/BLIP/output/Retrieval_aliproduct2_filtered/save_checkpoint_4.pth",
#     "model_size":"base","model_name":"base_1st_e_5"},
#     {"path":"/home/ubuntu/Desktop/base_2nd/base_2nd_e_5.pth","model_size":"base","model_name":"base_2nd_e_5"},
#     {"path":"/home/ubuntu/Desktop/base_3_itm/base_3rd_e_5.pth",
#     "model_size":"base","model_name":"base_3rd_e_5"},
#      {"path":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_8.pth","model_size":"large","model_name":"large_all_e_9"},
#     {"path":"/home/ubuntu/Desktop/large_v4/save_checkpoint_4.pth","model_size":"large","model_name":"large_v1_e_5"},
#     ]


models = [
       {"path":"/home/ubuntu/Desktop/large_v7/save_checkpoint_0.pth","model_size":"large","model_name":"large_v4_e_1"}
    ]

for model in models:
    print(f"extracting embeddings from "+model["model_name"])
    clip_model =ALIPRODUCT_BLIP(model["path"],image_size,vit=model["model_size"])
    image_embed,text_embed = gen_embed(col_to_test,clip_model)
    save_embed(image_embed,text_embed,model["model_size"],model["model_name"])
    


extracting embeddings from large_v4_e_1


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


load checkpoint from /home/ubuntu/Desktop/large_v7/save_checkpoint_0.pth


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting DataLoader 0: 100%|██████████| 313/313 [52:20<00:00, 10.03s/it]
saving img embed from large_v4_e_1 to ./image_embeds/val_set_embed/img_emded-large-large_v4_e_1.pkl
saving txt embed from large_v4_e_1 to ./text_embeds/val_set_embed/txt_emded-large-large_v4_e_1.pkl
